# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [16]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [19]:
# Function to convert date strings to datetime objects
def parse_date(date_str):
    for fmt in ("%m/%d/%Y", "%m/%d/%y", "%b-%d-%Y", "%b %d, %Y", "%B %d, %Y", "%B %d %Y", "%b %d %Y", "%b. %d, %Y","%B. %d, %Y", "%d %b %Y", "%d %B %Y", "%b %dth, %Y", "%B %dth, %Y", "%b %dst, %Y", "%B %dst, %Y", "%b %dnd, %Y", "%B %dnd, %Y", "%b %drd, %Y", "%B %drd, %Y", "%b %dth, %Y", "%B %dth, %Y", "%b %dst, %Y", "%B %dst, %Y", "%b %dnd, %Y", "%B %dnd, %Y", "%b %drd, %Y", "%B %drd, %Y", "%b %dth, %Y", "%B %dth, %Y", "%B %Y", "%b %Y", "%B, %Y", "%b, %Y"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT

def date_sorter():
    import re
    columnText = df.copy()
    df_result = pd.DataFrame({'text': columnText})
   
    df_result['text'] = df_result['text'].str.replace('Decemeber', 'December')

    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    pattern1 = r'\d{1,2}[/-]\d{1,2}[/-](?:\d{4}|\d{2})'

    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    pattern2 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)[.\-]?\s\d{1,2}[,-]?\s\d{4}'

    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    pattern3 = r'\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)[.,]?\s\d{4}'

    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    pattern4 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2}[st|nd|rd|th],\s\d{4}'

    # Feb 2009; Sep 2009; Oct 2010
    pattern5 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)[.,]?\s\d{4}'

    # 6/2008; 12/2009
    pattern6 = r'\d{1,2}/\d{4}'

    # 2009; 2010
    pattern7 = r'\d{4}'

    combined_pattern = f'{pattern1}|{pattern2}|{pattern3}|{pattern4}|{pattern5}|{pattern6}|{pattern7}'
    df_result['matched_dates'] = df_result['text'].str.findall(combined_pattern)
    df_result.to_excel('match.xlsx')
    dates = [d[1] if d[0] == '7787' else d[0] for d in df_result['matched_dates']]
    dates_result = []

    months = {
            '1': 'Jan',
            '2': 'Feb',
            '3': 'Mar',
            '4': 'Apr',
            '5': 'May',
            '6': 'Jun',
            '7': 'Jul',
            '8': 'Aug',
            '9': 'Sep',
            '10': 'Oct',
            '11': 'Nov',
            '12': 'Dec'
        }
    for date in dates:
        # If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
        if (bool(re.match(r'\b\d{1,2}/\d{4}\b', date))):
            d_split = date.split('/')
            # print(d_split)
            m = d_split[0]
            y = d_split[1]
            if (m.startswith('0')):
                m = m[1]
            # print(f'{months[m]} 1, {y}')
            dates_result.append(f'{months[m]} 1, {y}')

        # If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
        elif (bool(re.match(r'\b\d{4}\b', date))):
            dates_result.append(f'Jan 1, {date}')
        # Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)  
        elif (bool(re.match(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2}\b', date))):
            # print(date)
            d_split = re.split('/|-', date)
            # print(d_split)
            m = d_split[0]
            if (m.startswith('0')):
                m = m[1]
            d = d_split[1]
            if (d.startswith('0')):
                d = d[1]
            y = d_split[-1]
            ord = 'th'
            if d == '1':
                ord = 'st'
            elif d == '2':
                ord = 'nd'
            elif d == '3':
                ord = 'rd'
            # print(f'{months[m]} {d}, 19{y}')
            dates_result.append(f'{months[m]} {d}, 19{y}')
        else:
            dates_result.append(date)
    df_result['matched_dates'] = dates_result

    # Define a function to handle multiple date formats
    def parse_date(date_str):
        for fmt in ("%m/%d/%Y", "%m/%d/%y", "%b-%d-%Y", "%b %d, %Y", "%B %d, %Y", "%B %d %Y", "%b %d %Y", "%b. %d, %Y","%B. %d, %Y", "%d %b %Y", "%d %B %Y", "%b %dth, %Y", "%B %dth, %Y", "%b %dst, %Y", "%B %dst, %Y", "%b %dnd, %Y", "%B %dnd, %Y", "%b %drd, %Y", "%B %drd, %Y", "%b %dth, %Y", "%B %dth, %Y", "%b %dst, %Y", "%B %dst, %Y", "%b %dnd, %Y", "%B %dnd, %Y", "%b %drd, %Y", "%B %drd, %Y", "%b %dth, %Y", "%B %dth, %Y", "%B %Y", "%b %Y", "%B, %Y", "%b, %Y"):
            try:
                return pd.to_datetime(date_str, format=fmt)
            except ValueError:
                continue
        return pd.NaT

    # Apply the function to the DataFrame
    df_result['date'] = df_result['matched_dates'].apply(parse_date)
    # df_result.to_excel('result.xlsx')
    df_result.sort_values(by='date', inplace=True, kind='stable')
    order =  pd.Series(df_result.index.tolist())
    # df_result.to_excel('result_sorted.xlsx')
    # Your answer here
    return order

date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64